<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing<b> </font>

<font size="5"> <b>Lab 8: Change Detection in <font color='rgba(200,0,0,0.2)'>Your Own</font> SAR Amplitude Time Series Stack </b> </font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/>
</font>

<font size="3"> This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. It is introducing you to the methods of change detection in deep multi-temporal SAR image data stacks. Specifically, the lab applies the method of <i>Cumulative Sums</i> to perform change detection in a 60 image deep Sentinel-1 data stack over Niamey, Niger. As previously, the work will be done within the framework of a Jupyter Notebook.

<font color='rgba(200,0,0,0.2)'> <b>Note:</b> This version of Lab 8 is modified to allow for change detection analysis on your own data stack created within ASF HyP3</font> 
<br><br>

<b>In this chapter we introduce the following data analysis concepts:</b>

- How to use your own HyP3-generated data stack in a change detection effort
- The concepts of time series slicing by month, year, and date.
- The concepts and workflow of Cumulative Sum-based change point detection.
- The identification of change dates for each identified change point.
</font>

<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK INCLUDES NO HOMEWORK ASSIGNMENTS.</b></font> 

<font size="3">Contact me at fjmeyer@alaska.edu should you run into any problems.
</font>

</font>

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3"> The first step of this lab exercise on SAR image time series analysis is the import of necessary python libraries into your Jupyter Notebook. See the code cell below for information on which libraries are needed. Information on these libraries is provided in the instructions to a previous lab of this course (Lab 3). 

</font>

In [ ]:
import pandas as pd
import gdal
import numpy as np
import time,os

# For plotting
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.patches as patches

font = {'family' : 'monospace',
          'weight' : 'bold',
          'size'   : 18}
plt.rc('font',**font)

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Your Own Data Stack Into the Notebook </b> </font> 

<font size="3"> This lab assumes that you've created your own data stack over your personal area of interest using the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility's</a> value-added product system <a href="http://hyp3.asf.alaska.edu/" target="_blank">HyP3</a>. HyP3 is an environment that is used by ASF to prototype value added products and provide them to users to collect feedback. 

This lab expects Radiometric Terrain Corrected (RTC) image products as input. When creating your input data within HyP3, I recommend to stick to a unique orbit geometry (ascending or descending) to keep geometric differences between images low. 

We will retrieve HyP3 data via the HyP3 API. As both HyP3 and the Notebook environment sit in the <a href="https://aws.amazon.com/" target="_blank">Amazon Web Services (AWS)</a> cloud, data transfer is quick and cost effective.</font> 
</font>

<hr>
<font face="Calibri" size="3"> To download data from ASF, you need to provide your <a href="https://www.asf.alaska.edu/get-data/get-started/free-earthdata-account/" target="_blank">NASA Earth Data</a> username to the system. <b>The following field allows you to store your NASA Earth Data <font color='rgba(200,0,0,0.2)'>username and password</font> in this notebook for later use in data downloading:</b> </font>

In [ ]:
# Get URS creds and add to .netrc so we can do stuff!!
from getpass import getpass

NEDusr = 'fmeyer'
NEDpass = getpass()

with open('/home/jovyan/.netrc', 'w') as f:
    f.write('machine urs.earthdata.nasa.gov login ' + NEDusr + ' password ' + NEDpass + '\n')

<hr>
<font face="Calibri" size="3"> With your username/password now in place, you can now <b>log into the HyP3 API and query your existing subscriptions:</b> </font>

In [ ]:
# Login into the Hyp3 API
from asf_hyp3 import API
api = API(NEDusr)
api.login(NEDpass)

In [ ]:
# Get HyP3 Sub Name
subs = api.get_subscriptions()
#print(subs)
#xx = 10
for sub in subs:
    print(f"\nSubscription id: {sub['id']} {sub['name']}")

<hr>
<font face="Calibri" size="3"> Look through your existing subscriptions and <b>select the subscription ID</b> you would like to work with. <b>Enter the selected ID in the code cell below</b> (replace "1478" with the ID of your choice). </font>

In [ ]:
# Look at products in subscription and print out download urls
#products = api.get_products(sub_id=1478, page=0)
products = api.get_products(sub_id=1476)
print(products)

#print(f"\nURLs of products: \n")
for p in products:
    print(f"{p['url']}\n")

<hr>
<font face="Calibri" size="3"> Now you are ready to <b>download</b> all files associated with your subscription. </font>

In [ ]:
# Download products from subscription.
!mkdir -p granules

import re
for p in products:
    url = p['url']
    _match = re.match(r'https://hyp3-download.asf.alaska.edu/asf/data/(.*).zip', url)
    granule = _match.group(1)
    
    
    filename = 'granules/' + granule
    # Guess we need to download and unzip
    if not os.path.isdir(filename):
        print(f"Downloading new granule '{granule}' from '{url}'")
        !wget -O granules/{granule}.zip "{url}"
        !unzip -d granules granules/{granule}.zip
        !rm granules/{granule}.zip

<hr>
<font face="Calibri" size="3"> Run the following code cell to visualize the image acquisition dates in your subscription. </font>

In [ ]:
# We need dates for
!ls granules/*/*_VV.tif | sort -t_ -k5,5 | cut -c 27-34 > butte.dates
!cat butte.dates

<hr>
<font face="Calibri" size="3"> You may notice duplicates in your acquisition dates. As HyP3 processes SAR data on a frame-by-frame basis, duplicates may occur if your area of interest is covered by two consecutive  image frames. In this case, two separate images are generated that need to be merged together before time series processing can commence. <b>The next code cell is identifying frames in need to merging and is mosaicking these frames together.</b> </font>

In [ ]:
# Grab the paths of the VV
tiff_paths = !ls granules/*/*_VV.tif | sort -t_ -k5,5
print(f"Tiff paths: {tiff_paths}")

<hr>
<font face="Calibri" size="3"> Before you can merge frames, you need to <b>fix multiple UTM Zones-related issues</b> should they exist in your data set. If multiple UTM zones are fond, the following code cells will identify the predominant UTM zone and reproject the rest of your data stack into that zone. </font>

In [ ]:
!ls granules/*/*_VV.tif | sort -t_ -k5,5 > butte.files
if os.path.exists("test"):
    os.remove("test")
#!cat butte.files
dates=open('butte.dates').readlines()
files=open('butte.files').readlines()
output_file = ('utmzones.txt', 'w')
print('Checking UTM Zones in the data stack ...')
for  k in range(0, len(dates)):
    gdal_command = f"gdalinfo {tiff_paths[k]} | grep '^    AUTHORITY' | cut -d '\"' -f 2,4 | tr '\"' ':'"
    #print(f"Calling the command: {gdal_command}")
    !{gdal_command} >> test
    if ((k+1)/len(dates)*100)%5 == 0:
        print("%4.1f percent completed ..." % ((k+1)/len(dates)*100))
print('Done!')

In [ ]:
!cut -f 2 -d ':' test > utmzones
utmzones=[i.strip() for i in open('utmzones').readlines()]
utmzones2=[i.strip() for i in open('test').readlines()]

utmunique, counts = np.unique(utmzones, return_counts=True)
a = np.where(counts == np.max(counts))
maxutm = utmunique[a][0]
reproind = [i for i, j in enumerate(utmzones) if j != maxutm]
print('--------------------------------------------')
print('Reprojecting %4.1f files' %(len(reproind)))
print('--------------------------------------------')
for k in reproind:
    temppath = files[k].strip()
    _, granule_name, tiff_name = temppath.split('/')
    cmd = f"gdalwarp -overwrite granules/{granule_name}/{tiff_name} granules/{granule_name}/r{tiff_name} -s_srs {utmzones2[k]} -t_srs EPSG:{maxutm}"
    #print(f"Calling the command: {cmd}")
    !{cmd}
    rm_command = f"rm {files[k].strip()}"
    #print(f"Calling the command: {rm_command}")
    !{rm_command}

<hr>
<font face="Calibri" size="3"> Now you are ready to <b>concatenate neighboring image frames</b> should your area be covered by more than one frame. </font>

In [ ]:
!ls granules/*/*_VV.tif | sort -t_ -k5,5 > butte.files
#!cat butte.files
dates=open('butte.dates').readlines()
files=open('butte.files').readlines()
for  k in range(1, len(dates)):
    if dates[k] == dates[k-1]:
        #gdal_merge -o files[k-1] files[k] files[k-1]
        print(k)
        temp = tiff_paths[k-1]
        _, granule_name, tiff_name = temp.split('/')
        gdal_command = f"gdal_merge.py -o granules/{granule_name}/new-{tiff_name} {tiff_paths[k]} {tiff_paths[k-1]}"
        print(f"Calling the command: {gdal_command}")
        !{gdal_command}
        rm_command = f"rm {tiff_paths[k]}"
        print(f"Calling the command: {rm_command}")
        !{rm_command}
        rm_command = f"rm {tiff_paths[k-1]}"
        print(f"Calling the command: {rm_command}")
        !{rm_command}

<hr>
<font face="Calibri" size="3"> Let's verify that all date duplicates were resolved: </font>

In [ ]:
# We need dates for
!ls granules/*/*_VV.tif | sort -t_ -k5,5 | cut -c 27-34 > butte.dates
!cat butte.dates

<hr>
<font face="Calibri">

<font size="5"> <b> 2. Create Subset and Stack Up Your Data </b> </font> 

<font size="3"> Now you are ready to work with your data. The next cells allow you to select an area of interest (AOI; via bounding-box corner coordinates) for your data analysis. Once selected, the AOI is being extracted and a data stack is formed.

<b>As a first step, we extract your AOI from the full frames:</b>
</font> 
</font>

In [ ]:
# Using Google Maps, get the rough bounding box for the subset
ulx = -121.65
lrx = -121.4
uly = 39.85
lry = 39.7
!echo {ulx} {lrx} {lry} {uly}

In [ ]:
# Grab the paths of the VV
tiff_paths = !ls granules/*/*_VV.tif | sort  -t_ -k5,5
#print(f"Tiff paths: {tiff_paths}")

In [ ]:
# Cycle through and subset the tiffs in the products
!mkdir -p tiffs
for tiff_path in tiff_paths:
    _, granule_name, tiff_name = tiff_path.split('/')
    g1, g2, g3, date, g4, g5, g6 = tiff_name.split('_')
    # Using the GDAL subset service, get a small subset around the Butte
    #!wget -O {granule_name}_VV.tiff "https://services.asf.alaska.edu/geospatial/subset?ulx={ulx}&lrx={lrx}&lry={lry}&uly={uly}&product={granule_name}.zip/{granule_name}/{tiff_name}"

    # GDAL service is out of service. Pretend that it isn't when calling the following equivalent command
    gdal_command = f"gdal_translate -projwin {ulx} {uly} {lrx} {lry} -projwin_srs 'WGS84' -co \"COMPRESS=DEFLATE\" -a_nodata 0 {tiff_path} tiffs/{date}_VV.tiff"
    print(f"Calling the command: {gdal_command}")
    !{gdal_command}

<hr>
<font face="Calibri" size="3"> Now we stack up your data by creating a virtual raster table with links to all subset data files: </font>

In [ ]:
# Create the VRT for the downloaded subset geotiffs
# Grab all tiffs in the directory
!gdalbuildvrt -separate butte.vrt tiffs/*.tiff

In [ ]:
# We need dates for
!ls tiffs/*_VV.tiff | sort | cut -c 7-21 > butte.dates
!cat butte.dates

<hr>
<font face="Calibri">

<font size="5"> <b> 3. Now You Can Work With Your Data </b> </font> 

<font size="3"> Now you are ready to perform time series change detection on your data stack.
</font> 
</font>

<br>
<font face="Calibri" size="4"> <b> 3.1 Define Data Directory and Path to VRT </b> </font> 

<font face="Calibri" size="3"> Just some path definitions. </font>

In [ ]:
# Set some paths
datadirectory='/home/jovyan/notebooks/ASF/GEOS_657_Labs'
datefile='butte.dates'
imagefile='butte.vrt'

In [ ]:
# Get some indices for plotting
dates=open(datefile).readlines()
tindex=pd.DatetimeIndex(dates)

In [ ]:
# Bands and times
j=1
print('Bands and dates for',imagefile)
for i in tindex:
    print("{:4d} {}".format(j, i.date()),end=' ')
    j+=1
    if j%5==1: print()

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.2 Open Your Data Stack and Visualize Some Layers </b> </font> 

<font face="Calibri" size="3"> We will open your VRT and visualize some layers using Matplotlib. </font>

In [ ]:
# Open virtual dataset
img=gdal.Open(imagefile)

In [ ]:
print(img.RasterCount) # Number of Bands
print(img.RasterXSize) # Number of Pixels
print(img.RasterYSize) # Number of Lines

In [ ]:
# Read in raster data for the first two bands
raster_1 = img.GetRasterBand(1).ReadAsArray()
where_are_NaNs = np.isnan(raster_1)
raster_1[where_are_NaNs] = 0

raster_3 = img.GetRasterBand(16).ReadAsArray()
where_are_NaNs = np.isnan(raster_3)
raster_3[where_are_NaNs] = 0

In [ ]:
# Plot some things
fig = plt.figure(figsize=(18,10)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 122 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot

# First plot: Image
bandnbr=1
ax1.imshow(raster_1,cmap='gray',vmin=0,vmax=0.2) #,vmin=2000,vmax=10000)
ax1.set_title('Image Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))

# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax2.hist(raster_1.flatten(),bins=200,range=(0,0.3))
ax2.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax2.set_title('Histogram Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))


# Third plot: Image
bandnbr=2
ax3.imshow(raster_3,cmap='gray',vmin=0,vmax=0.2) #,vmin=2000,vmax=10000)
ax3.set_title('Image Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))

# Fourth plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax4.hist(raster_3.flatten(),bins=200,range=(0,0.3))
ax4.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax4.set_title('Histogram Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.3 Create a Time Series Animation </b> </font>

<font face="Calibri" size="3"> Now we are ready to <b>create a time series animation</b> from the calibrated SAR data. </font> 

In [ ]:
band = img.GetRasterBand(1)

In [ ]:
raster0 = band.ReadAsArray()
bandnbr=0 # Needed for updates
rasterstack=img.ReadAsArray()

In [ ]:
rs2 = np.ma.masked_where(rasterstack == 0, rasterstack)

In [ ]:
%%capture 
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np

fig=plt.figure(figsize=(14,8))
ax = fig.add_subplot(111)
ax.axis('off')
vmin=np.percentile(rasterstack.flatten(),5)
vmax=np.percentile(rasterstack.flatten(),95)

r0dB=20*np.log10(raster0)-83

im = ax.imshow(raster0,cmap='gray',vmin=vmin,vmax=vmax)
ax.set_title("{}".format(tindex[0].date()))

def animate(i):
    ax.set_title("{}".format(tindex[i].date()))
    im.set_data(rasterstack[i])

# Interval is given in milliseconds
ani = matplotlib.animation.FuncAnimation(fig, animate, 
                                         frames=rasterstack.shape[0],
                                        interval=400)

In [ ]:
from matplotlib import animation, rc
rc('animation',embed_limit=40971520.0)  # We need to increase the 
            # limit maybe to show the entire animation

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
ani.save('animation.gif', writer='pillow', fps=2)

<br>
<hr>
<font face="Calibri" size="5"> <b> 4. Cummulative Sum-based Change Detection Across an Entire Image</b> </font> 

<font face="Calibri" size="3"> With numpy arrays we can apply the concept of **cumulative sum change detection** analysis effectively on the entire image stack. We take advantage of array slicing and axis-based computing in numpy. Axis 0 is the time domain in our raster stacks.
    
<hr>
<font size="4"><b>4.1 We first create our time series stack:</b>
</font> 

In [ ]:
X= 10.*np.log10(rs2)  # Uncomeent to test dB scale 

<font face="Calibri" size="3">Sometimes it makes sense to <b>extract a reduced time span</b> from the full time series to reduce the number of different change objects in a scene. In the following, we extract a shorter time span:
</font>

In [ ]:
dind = np.where((tindex >'2017-10-20') & (tindex <'2018-10-31'))
X_sub=np.squeeze(X[dind,:,:])
tindex_sub=tindex[dind]

In [ ]:
X_sub.shape

In [ ]:
plt.figure(figsize=(12, 8))
bandnbr=0
vmin=np.percentile(X_sub[bandnbr],5)
vmax=np.percentile(X_sub[bandnbr],95)
plt.title('Band  {} {}'.format(bandnbr+1,tindex_sub[bandnbr].date()))
plt.imshow(X_sub[0],cmap='gray',vmin=vmin,vmax=vmax)
_=plt.colorbar()

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.2 Calculate Mean Across Time Series to Prepare for Calculation of Cummulative Sum $S$:</b> </font> 

In [ ]:
Xmean=np.mean(X_sub,axis=0)
plt.figure(figsize=(12, 8))
plt.imshow(Xmean,cmap='gray')
_=plt.colorbar()

In [ ]:
R=X_sub-Xmean

In [ ]:
plt.figure(figsize=(12, 8))
plt.imshow(R[0])
plt.title('Residuals for Band  {} {}'.format(bandnbr+1,tindex_sub[bandnbr].date()))
_=plt.colorbar()

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.3 Calculate Cummulative Sum $S$ as well as Change Magnitude $S_{diff}$:</b> </font> 

In [ ]:
S = np.cumsum(R,axis=0)
Smax= np.max(S,axis=0)
Smin= np.min(S,axis=0)
Sdiff=Smax-Smin
fig,ax=plt.subplots(1,3,figsize=(16,4))
vmin=np.percentile(Smin.flatten(),3)
vmax=np.percentile(Smax.flatten(),97)
p=ax[0].imshow(Smax,vmin=vmin,vmax=vmax)
ax[0].set_title('$S_{max}$')
ax[1].imshow(Smin,vmin=vmin,vmax=vmax)
ax[1].set_title('$S_{min}$')
ax[2].imshow(Sdiff,vmin=vmin,vmax=vmax)
ax[2].set_title('$S_{diff}$')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
_=fig.colorbar(p,cax=cbar_ax)

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.4 Mask $S_{diff}$ With a-priori Threshold To Idenfity Change Candidates:</b> </font>

<font face="Calibri" size="3">To identified change candidate pixels, we can threshold $S_{diff}$ to reduce computation of the bootstrapping. For land cover change, we would not expect more than 5-10% change pixels in a landscape. So, if the test region is reasonably large, setting a threshold for expected change to 10% is appropriate. In our example, we'll start out with a very conservative threshold of 50%.

The histogram for $S_{diff}$ is shown below.
</font>

In [ ]:
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(14,6)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(122)
# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax1.hist(Sdiff.flatten(),bins=200,range=(0,np.max(Sdiff)))
ax1.xaxis.set_label_text('Change Magnitude')
ax1.set_title('Change Magnitude Histogram')
plt.grid()
n, bins, patches = ax2.hist(Sdiff.flatten(), bins=200, range=(0,np.max(Sdiff)), cumulative='True', density='True', histtype='step', label='Empirical')
ax2.xaxis.set_label_text('Change Magnitude')
ax2.set_title('Change Magnitude CDF')
plt.grid()

In [ ]:
precentile=0.9
outind = np.where(n > precentile)
threshind = np.min(outind)
thres = bins[threshind]
print('At the {}% percentile, the threshold value is {:2.2f}'.format(precentile*100,thres))

<font face="Calibri" size="3">Using this threshold, we can <b>visualize our change candidate areas</b>:
</font>

In [ ]:
Sdiffmask=Sdiff<thres
plt.figure(figsize=(12, 8))
plt.title('Change Candidate Areas (black)')
_=plt.imshow(Sdiffmask,cmap='gray')

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.5 Bootstrapping to Prepare for Change Point Selection:</b> </font>

<font face="Calibri" size="3">We can now perform bootstrapping over the candidate pixels. The workflow is as follows:
<ul>
    <li>Filter our residuals to the change candidate pixels</li>
    <li>Perform bootstrapping over candidate pixels</li>
</ul>
For efficient computing we permutate the index of the time axis.
</font>

In [ ]:
Rmask = np.broadcast_to(Sdiffmask,R.shape)
Rmasked = np.ma.array(R,mask=Rmask)

<font face="Calibri" size="3">On the masked time series stack of residuals, we can re-compute the cumulative sums:
</font>

In [ ]:
Smasked = np.ma.cumsum(Rmasked,axis=0)

In [ ]:
Smasked_max= np.ma.max(Smasked,axis=0)
Smasked_min= np.ma.min(Smasked,axis=0)
Smasked_diff=Smasked_max-Smasked_min
fig,ax=plt.subplots(1,3,figsize=(16,4))
vmin=Smasked_min.min()
vmax=Smasked_max.max()
p=ax[0].imshow(Smasked_max,vmin=vmin,vmax=vmax)
ax[0].set_title('$S_{max}$')
ax[1].imshow(Smasked_min,vmin=vmin,vmax=vmax)
ax[1].set_title('$S_{min}$')
ax[2].imshow(Smasked_diff,vmin=vmin,vmax=vmax)
ax[2].set_title('$S_{diff}$')
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
_=fig.colorbar(p,cax=cbar_ax)

<font face="Calibri" size="3">Now let's perform <b>bootstrapping</b>:
</font>

In [ ]:
random_index=np.random.permutation(Rmasked.shape[0])
Rrandom=Rmasked[random_index,:,:]

In [ ]:
n_bootstraps=100  # bootstrap sample size

# to keep track of the maxium Sdiff of the bootstrapped sample:
Sdiff_random_max = np.ma.copy(Smasked_diff) 
Sdiff_random_max[~Sdiff_random_max.mask]=0
# to compute the Sdiff sums of the bootstrapped sample:
Sdiff_random_sum = np.ma.copy(Smasked_diff) 
Sdiff_random_sum[~Sdiff_random_max.mask]=0
# to keep track of the count of the bootstrapped sample
n_Sdiff_gt_Sdiff_random = np.ma.copy(Smasked_diff) 
n_Sdiff_gt_Sdiff_random[~n_Sdiff_gt_Sdiff_random.mask]=0
print("Running Bootstrapping for %4.1f iterations ..." % (n_bootstraps))
for i in range(n_bootstraps):
    # For efficiency, we shuffle the time axis index and use that 
    #to randomize the masked array
    random_index=np.random.permutation(Rmasked.shape[0])
    # Randomize the time step of the residuals
    Rrandom = Rmasked[random_index,:,:]  
    Srandom = np.ma.cumsum(Rrandom,axis=0)
    Srandom_max=np.ma.max(Srandom,axis=0)
    Srandom_min=np.ma.min(Srandom,axis=0)
    Sdiff_random=Srandom_max-Srandom_min
    Sdiff_random_sum += Sdiff_random
    Sdiff_random_max[np.ma.greater(Sdiff_random,Sdiff_random_max)]=\
    Sdiff_random[np.ma.greater(Sdiff_random,Sdiff_random_max)]
    n_Sdiff_gt_Sdiff_random[np.ma.greater(Smasked_diff,Sdiff_random)] += 1
    if ((i+1)/n_bootstraps*100)%10 == 0:
        print("%4.1f percent completed ..." % ((i+1)/n_bootstraps*100))

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.6 Extract Confidence Metrix and Select Final Change Points:</b> </font>

<font face="Calibri" size="3">We first compute for all pixels the confidence level $CL$, the change point significance metric $CP_{significance}$ and the product of the two as our confidence metric for identified change points:
</font>

In [ ]:
CL = n_Sdiff_gt_Sdiff_random/n_bootstraps
CP_significance = 1.- (Sdiff_random_sum/n_bootstraps)/Sdiff 
#Plot
fig,ax=plt.subplots(1,3,figsize=(16,4))
a = ax[0].imshow(CL*100)
fig.colorbar(a,ax=ax[0])
ax[0].set_title('Confidence Level %')
a = ax[1].imshow(CP_significance)
fig.colorbar(a,ax=ax[1])
ax[1].set_title('Significance')
a = ax[2].imshow(CL*CP_significance)
fig.colorbar(a,ax=ax[2])
_=ax[2].set_title('CL x S')

<font face="Calibri" size="3">Now we can set a change point threshold to identify most likely change pixels in our map of change candidates:
</font>

In [ ]:
cp_thres=0.25

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1,1,1)
plt.title('Detected Change Pixels based on Threshold %2.1f' % (cp_thres))
a = ax.imshow(CL*CP_significance <  cp_thres,cmap='cool')

<br>
<hr>
<font face="Calibri" size="4"> <b> 4.7 Derive Timing of Change for Each Change Pixel:</b> </font>

<font face="Calibri" size="3">Our last step in the identification of the change points is to extract the timing of the change. We will produce a raster layer that shows the band number of this first date after a change was detected. We will make use of the numpy indexing scheme. First, we create a combined mask of the first threshold and the identified change points after the bootstrapping. For this we use the numpy "mask_or" operation.
</font>

In [ ]:
# make a mask of our change points from the new threhold and the previous mask
cp_mask=np.ma.mask_or(CL*CP_significance<cp_thres,CL.mask)
# Broadcast the mask to the shape of the masked S curves
cp_mask2 = np.broadcast_to(cp_mask,Smasked.shape)
# Make a numpy masked array with this mask
CPraster = np.ma.array(Smasked.data,mask=cp_mask2)

<font face="Calibri" size="3">To retrieve the dates of the change points we find the band indices in the time series along the time axis where the maximum of the cumulative sums was located. Numpy offers the "argmax" function for this purpose.
</font>

In [ ]:
CP_index= np.ma.argmax(CPraster,axis=0)
change_indices = list(np.unique(CP_index))
change_indices.remove(0)
print(change_indices)
# Look up the dates from the indices to get the change dates
alldates=tindex_sub
change_dates=[str(alldates[x].date()) for x in change_indices]
print(change_dates)

<font face="Calibri" size="3">Lastly, we visualize the change dates by showing the $CP_{index}$ raster and label the change dates.
</font>

In [ ]:
ticks=change_indices
ticklabels=change_dates

cmap=plt.cm.get_cmap('tab20',ticks[-1])
fig, ax = plt.subplots(figsize=(12,12))
cax = ax.imshow(CP_index,interpolation='nearest',cmap=cmap)
# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(p,cax=cbar_ax)

ax.set_title('Dates of Change')
# cbar = fig.colorbar(cax,ticks=ticks)
cbar=fig.colorbar(cax,ticks=ticks,orientation='horizontal')
_=cbar.ax.set_xticklabels(ticklabels,size=10,rotation=45,ha='right')  

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 1.0 - April 2019 </i>
</font>